[link do tutorialu](https://www.analyticsvidhya.com/blog/2022/03/music-genre-classification-project-using-machine-learning-techniques/)

# K Nearest Neighbors (KNN) classification algorithm
Jest to podobno najlepiej sprawdzająca się metoda klasyfikacji dla ścieżek dźwiękowych.

## Instalacja bibliotek

In [1]:
!pip install python_speech_features # biblioteka do ekstrakcji cech
!pip install scipy  # biblioteka do przetwarzania plików dźwiękowych (w tym wypadku do odczytu plików .wav)

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5879 sha256=85950fdf0e07213543aa106a8d010ea57419e350aa100033677f361fa48c9ca9
  Stored in directory: c:\users\dominik dębiński\appdata\local\pip\cache\wheels\37\01\19\e6c69a32684ab7b2e3ea4985a571d810cf055c72600e7f9f17
Successfully built python_speech_features


## Import bibliotek

In [2]:
import numpy as np
import pandas as pd
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tempfile import TemporaryFile
import os
import math
import pickle
import random
import operator

In [3]:
#define a function to get distance between feature vectors and find neighbors
def getNeighbors(trainingset, instance, k):
    distances = []
    for x in range(len(trainingset)):
        dist = distance(trainingset[x], instance, k) + distance(instance,trainingset[x],k)
        distances.append((trainingset[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [4]:
#function to identify the nearest neighbors
def nearestclass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1

    sorter = sorted(classVote.items(), key=operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [5]:
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == prediction[x]:
            correct += 1
    return 1.0 * correct / len(testSet)

In [9]:
generes_directory = '../WTUM_gr12/Data/genres_original'
f = open("mydataset.dat", "wb")
i = 0
for folder in os.listdir(generes_directory):
    # print(folder)
    i += 1
    if i == 11:
        break
    for file in os.listdir(generes_directory + "/" + folder):
        # print(file)
        try:
            (rate, sig) = wav.read(generes_directory + "/" + folder + "/" + file)
            mfcc_feat = mfcc(sig, rate, winlen = 0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print("Got an exception: ", e, 'in folder: ', folder, ' filename: ', file)
f.close()

Got an exception:  File format b'\xcb\x15\x1e\x16' not understood. Only 'RIFF' and 'RIFX' supported. in folder:  jazz  filename:  jazz.00054.wav


In [12]:
dataset = []
dataset_file_name = 'mydataset.dat'
def loadDataset(filename, split, trset, teset):
    with open(dataset_file_name ,'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trset.append(dataset[x])
        else:
            teset.append(dataset[x])

trainingSet = []
testSet = []
loadDataset(dataset_file_name, 0.68, trainingSet, testSet)

In [13]:
def distance(instance1, instance2, k):
    distance = 0
    mm1 = instance1[0]
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    distance += (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
    distance += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance -= k
    return distance

In [14]:
# Make the prediction using KNN(K nearest Neighbors)
length = len(testSet)
predictions = []
for x in range(length):
    predictions.append(nearestclass(getNeighbors(trainingSet, testSet[x], 5)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

0.7012578616352201


In [16]:
from collections import defaultdict
results = defaultdict(int)

generes_directory = '../WTUM_gr12/Data/genres_original'
i = 1
for folder in os.listdir(generes_directory):
    results[i] = folder
    i += 1

In [19]:
results

defaultdict(int,
            {1: 'blues',
             2: 'classical',
             3: 'country',
             4: 'disco',
             5: 'hiphop',
             6: 'jazz',
             7: 'metal',
             8: 'pop',
             9: 'reggae',
             10: 'rock'})

In [20]:
pred = nearestclass(getNeighbors(dataset, feature, 5))
print(pred)
print(results[pred])

10
rock
